In [2]:
import torch

In [ ]:
#Sea x una entrada de dimensión 512 con distribución N(0,1)
# Media: 0
# Varianza: 1

x = torch.randn(512)

#Simular la pasada forward de la entrada con 100 capas lineales
#Las capas tienen 512 neuronas cada una

for i in range(100):
    w = torch.randn(512,512)
    x = w @ x
x.mean(), x.std()

#Media y desviación del resultado explotan

In [ ]:
# En qué capa sucede la explosión?

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512)
    x = a @ x
    if torch.isnan(x.std()): break

print(i)

#Entrada es pequeña, la única razón para la explosión es que los pesos son muy grandes

In [ ]:
#Podemos vernos tentados a reducir los pesos para evitar la explosión
# Escalamos los pesos por algún factor

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * 0.01
    x = a @ x
x.mean(), x.std()

# Ahora la media y la desviación se fueron a cero

In [ ]:
#Cuál es el promedio y desviación estándar de multiplicar un vector de 512 dimensiones y una matriz 512x512?
# Ambos en N(0,1)

#Ejecutamos 10000 multiplicaciones, y promediamos los resultados

import math

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

In [ ]:
#La desviación es muy similar a la raiz cuadrada de la dimension del vector de entrada

math.sqrt(512)

In [ ]:
#El producto de dos números en distribucipón N(0,1) es siempre un número en la misma distribución

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, math.sqrt(var/10000)


In [ ]:
#La varianza promedio debe estar en el orden de 1/512
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(1)
    a = torch.randn(1)*math.sqrt(1./512)
    y = a*x
    mean += y.item()
    var += y.pow(2).item()
mean/10000, var/10000

In [ ]:
1/512

In [ ]:
#Así que deberíamos usar sqrt(1/512) para escalar los pesos

mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(1./512)
    y = a @ x
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

In [ ]:
#Probemos en nuestra red neuronal simulada

x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = a @ x
x.mean(), x.std()

#Las salidas no explotan ni se desvanecen

In [13]:
#Hasta ahora no hemos utilizado funciones de activación.
#Veamos que pasa si aplicamos una función de activación
#TANH a nuestro modelo basico de red neuronal

def tanh(x): return torch.tanh(x)

In [ ]:
x = torch.randn(512)

for i in range(100):
    a = torch.randn(512,512) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

In [ ]:
x = torch.randn(512)

for i in range(100):
    a = torch.Tensor(512,512).uniform_(-1, 1) * math.sqrt(1./512)
    x = tanh(a @ x)
x.mean(), x.std()

In [ ]:
#Glorot y Bengio propusieron una nueva inicialización
def xavier(m,h):
    return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

x = torch.randn(512)

for i in range(100):
    a = xavier(512, 512)
    x = tanh(a @ x)
x.mean(), x.std()

In [17]:
#Pero que pasa cuando la función de activación es RELU?
def relu(x): return x.clamp_min(0.)

In [ ]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

In [ ]:
#Desviación estándar es cernaca a sqrt(512)/sqrt(2)
math.sqrt(512/2)

In [ ]:
mean, var = 0.,0.
for i in range(10000):
    x = torch.randn(512)
    a = torch.randn(512,512)*math.sqrt(2/512)
    y = relu(a @ x)
    mean += y.mean().item()
    var += y.pow(2).mean().item()
mean/10000, math.sqrt(var/10000)

In [ ]:
def kaiming(m,h):
  return torch.randn(m,h)*math.sqrt(2./m)

x = torch.randn(512)

for i in range(100):
  a = kaiming(512, 512)
  x = relu(a @ x)

x.mean(), x.std()

In [ ]:
#Xavier con RELU?

x = torch.randn(512)

for i in range(100):
  a = xavier(512, 512)
  x = relu(a @ x)

x.mean(), x.std()